# Create High-k Metal Gate Stack Tutorial

This notebook demonstrates how to create a high-k metal gate stack heterostructure with four materials: Si (substrate), SiO2 (gate oxide), HfO2 (high-k dielectric), and TiN (metal gate).


## 1. Configuration Parameters


In [ ]:
# Global parameters
MAX_AREA = 800  # Maximum area for all interfaces
MAX_AREA_RATIO_TOL = 0.09  # Maximum area ratio tolerance for strain matching
MAX_ANGLE_TOLERANCE = 0.03  # Maximum angle tolerance for strain matching
MAX_LENGTH_TOLERANCE = 0.03  # Maximum length tolerance for strain matching
FINAL_VACUUM = 20.0  # Final vacuum spacing in Angstroms

# Structure parameters for each layer
STRUCTURE_PARAMS = [
    {
        # Silicon substrate
        "slab_params": {
            "miller_indices": (1, 0, 0),
            "thickness": 3,  # atomic layers
            "vacuum": 0,  # Angstroms
            "xy_supercell_matrix": [[1, 0], [0, 1]],
            "use_orthogonal_z": True
        },
        "interface_distance": None  # No interface for substrate
    },
    {
        # SiO2 layer
        "slab_params": {
            "miller_indices": (1,0,0),
            "thickness": 2,
            "vacuum": 0,
            "xy_supercell_matrix": [[1, 0], [0, 1]],
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to Si substrate
    },
    {
        # HfO2 layer
        "slab_params": {
            "miller_indices": (0, 1, 1),
            "thickness": 3,
            "vacuum": 0,
            "xy_supercell_matrix": [[1, 0], [0, 1]],
            "use_orthogonal_z": True
        },
        "interface_distance": 2.8  # Distance to SiO2
    },
    {
        # TiN layer
        "slab_params": {
            "miller_indices": (0, 0, 1),
            "thickness": 3,
            "vacuum": FINAL_VACUUM,  # Add vacuum to final layer
            "xy_supercell_matrix": [[1, 0], [0, 1]],
            "use_orthogonal_z": True
        },
        "interface_distance": 2.5  # Distance to HfO2
    }
]

INTERFACE_INDEX=[4,0,0]


## 2. Environment Setup and Material Loading


In [ ]:
from mat3ra.standata.materials import Materials
from mat3ra.made.material import Material
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab
from mat3ra.made.tools.build.interface import (
    InterfaceConfiguration,
    ZSLStrainMatchingParameters,
    ZSLStrainMatchingInterfaceBuilder,
    ZSLStrainMatchingInterfaceBuilderParameters
)
from mat3ra.made.tools.modify import translate_to_z_level
from utils.visualize import visualize_materials as visualize

# Load materials from Standata
materials = [
    Material(Materials.get_by_name_first_match("Silicon")),  # Si substrate
    Material(Materials.get_by_name_first_match("SiO2")),  # SiO2
    Material(Materials.get_by_name_first_match("HfO2")),  # HfO2
    Material(Materials.get_by_name_first_match("TiN"))  # TiN
]


## 3. Build Stack Layer by Layer

Now we'll build the stack sequentially, starting with the substrate and adding each layer one by one.


In [ ]:
# Start with substrate (first material)
current_structure = materials[0]
print("Starting with substrate:", materials[0].name)

# Iterate through remaining materials to build interfaces
for i in range(1, len(materials)):
    print(f"\nBuilding interface {i} of {len(materials)-1}...")
    print(f"Adding {materials[i].name} to {materials[i-1].name}")
    
    # Get parameters for current interface
    film_params = STRUCTURE_PARAMS[i]
    substrate_params = STRUCTURE_PARAMS[i-1]
    
    # Create slab configurations
    film_config = SlabConfiguration(
        bulk=materials[i],
        **film_params["slab_params"]
    )
    
    substrate_config = SlabConfiguration(
        bulk=current_structure,
        **substrate_params["slab_params"]
    )
    
    # Get terminations
    film_terminations = get_terminations(film_config)
    substrate_terminations = get_terminations(substrate_config)
    
    # Use first termination pair for simplicity
    film_termination = film_terminations[0]
    substrate_termination = substrate_terminations[0]
    
    print(f"Using terminations: {film_termination} (film) and {substrate_termination} (substrate)")
    
    # Create interface configuration
    interface_config = InterfaceConfiguration(
        film_configuration=film_config,
        substrate_configuration=substrate_config,
        film_termination=film_termination,
        substrate_termination=substrate_termination,
        distance=film_params["interface_distance"],
        vacuum=film_params["slab_params"]["vacuum"]
    )
    
    # Set up strain matching and build interface
    strain_params = ZSLStrainMatchingParameters(max_area=MAX_AREA, max_area_ratio_tol=MAX_AREA_RATIO_TOL, max_angle_tol=MAX_ANGLE_TOLERANCE, max_length_tol=MAX_LENGTH_TOLERANCE)
    builder = ZSLStrainMatchingInterfaceBuilder(
        build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
            strain_matching_parameters=strain_params
        )
    )
    
    # Generate and sort interfaces
    interfaces = builder.get_materials(configuration=interface_config)
    
    # Select the first interface (lowest strain, smallest area)
    current_structure = interfaces[INTERFACE_INDEX[i-1]]
    
    # Translate structure to prepare for next layer
    # current_structure = translate_to_z_level(current_structure, "top")
    
    # Visualize current state
    visualize(
        current_structure,
        repetitions=[1, 1, 1],
        title=f"After adding {materials[i].name}"
    )
    
    # # Print interface statistics
    # print(f"\nLayer {i} Statistics:")
    # print(f"Number of atoms: {len(current_structure.atoms)}")
    print(f"Height: {current_structure.lattice.c:.2f} Å")
    print(f"Surface area: {current_structure.lattice.volume()/current_structure.lattice.c:.2f} Å²")


## 4. Final Structure Visualization and Analysis


In [ ]:
# Visualize final structure
visualize(
    current_structure,
    repetitions=[3, 3, 1],
    title="Complete High-k Metal Gate Stack"
)

# Print final analysis
print("\nFinal Structure Analysis:")
print(f"Total number of atoms: {len(current_structure.atoms)}")
print(f"Total height: {current_structure.lattice.c:.2f} Å")
print(f"Surface area: {current_structure.lattice.surface_area:.2f} Å²")
print("\nLayer composition:")
for element, count in current_structure.composition.items():
    print(f"  {element}: {count} atoms")